# Create your custom types with Pixano [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pixano/pixano/blob/main/notebooks/types/create_custom_type.ipynb)

This notebook will help you create custom data types for your datasets.

This will allow you to store your data exactly the way you want with Pixano.

## 1. Setting up

### Install dependencies

This notebook requires installing `pixano`.

If you are running this notebook on your computer, we strongly recommend creating a virtual environment for using Pixano like so:

```shell
conda create -n pixano_env python=3.10
conda activate pixano_env
```

```shell
pip install pixano
```

If you are running this notebook in Google Colab, run the cell below to install `pixano`.


In [ ]:
try:
    import google.colab

    ENV = "colab"
    !pip install pixano
except:
    ENV = "jupyter"

### Load dependencies


In [ ]:
import tempfile
from typing import Optional
from pathlib import Path

import lance
import lancedb
import pyarrow as pa
from pydantic import BaseModel

from pixano.core import BBox, BBoxType, PixanoType, create_pyarrow_type

## 2. Creating a type

- Use `pixano.core.PixanoType` and `pydantic.BaseModel` as parent classes
- Define your attributes and methods
- Override the `to_struct` method to define what to export inside your PyArrow format dataset files

Note that PixanoType provides you with generic `from_dict` and `to_dict` methods based on the fields you define in `to_struct`.


In [ ]:
class Camera(PixanoType, BaseModel):
    """Camera type

    Attributes:
        depth_scale (float): Depth scale
        cam_K (list[float]): Camera matrix K
        cam_R_w2c (list[float], optional): 3*3 orientation matrix
        cam_t_w2c (list[float], optional): 3*1 translation matrix
    """

    depth_scale: float
    cam_K: list[float]
    cam_R_w2c: Optional[list[float]]
    cam_t_w2c: Optional[list[float]]

    def __init__(
        self,
        depth_scale: float,
        cam_K: list[float],
        cam_R_w2c: list[float] = None,
        cam_t_w2c: list[float] = None,
    ):
        """Initialize Camera

        Args:
            depth_scale (float): Depth scale
            cam_K (list[float]): Camera matrix K
            cam_R_w2c (list[float], optional): 3*3 orientation matrix. Defaults to None.
            cam_t_w2c (list[float], optional): 3*1 translation matrix. Defaults to None.
        """

        # Define public attributes through Pydantic BaseModel
        super().__init__(
            depth_scale=depth_scale,
            cam_K=cam_K,
            cam_R_w2c=cam_R_w2c,
            cam_t_w2c=cam_t_w2c,
        )

    @staticmethod
    def to_struct() -> pa.StructType:
        """Return Camera type as PyArrow Struct

        Returns:
            pa.StructType: Custom type corresponding PyArrow Struct
        """

        return pa.struct(
            [
                pa.field("depth_scale", pa.float64()),
                pa.field("cam_K", pa.list_(pa.float64())),
                pa.field("cam_R_w2c", pa.list_(pa.float64())),
                pa.field("cam_t_w2c", pa.list_(pa.float64())),
            ]
        )

Test your newly created type to make sure it works:


In [31]:
camera_example = Camera.from_dict(
    {
        "cam_K": [
            1758.377685546875,
            0.0,
            360.0000000121072,
            0.0,
            1781.137258093513,
            269.9999999622624,
            0.0,
            0.0,
            1.0,
        ],
        "cam_R_w2c": [
            -0.8058909773826599,
            -0.5643280148506165,
            -0.17909124493598938,
            -0.5611616969108582,
            0.8244928121566772,
            -0.0728636085987091,
            0.18877841532230377,
            0.04177902266383171,
            -0.9811305999755859,
        ],
        "cam_t_w2c": [
            -10.521206855773926,
            40.88941192626953,
            1092.1990966796875,
        ],
        "depth_scale": 0.1,
    }
)

In [32]:
camera_example.to_dict()

{'depth_scale': 0.1,
 'cam_K': [1758.377685546875,
  0.0,
  360.0000000121072,
  0.0,
  1781.137258093513,
  269.9999999622624,
  0.0,
  0.0,
  1.0],
 'cam_R_w2c': [-0.8058909773826599,
  -0.5643280148506165,
  -0.17909124493598938,
  -0.5611616969108582,
  0.8244928121566772,
  -0.0728636085987091,
  0.18877841532230377,
  0.04177902266383171,
  -0.9811305999755859],
 'cam_t_w2c': [-10.521206855773926, 40.88941192626953, 1092.1990966796875]}

The PyArrow type will be used to store your data inside the lance files Pixano uses.

Thanks to the `to_struct` method you just defined, all you have to do is call `createPyArrowType` to create your type's corresponding PyArrow type.


In [ ]:
CameraType = create_pyarrow_type(Camera.to_struct(), "Camera", Camera)

## 3. Using the type

Here is a list of things you can do to use your types and make sure everything is setup properly.

Define a PyArrow schema using your custom type alongside other PixanoTypes:


In [ ]:
schema = pa.schema(
    [
        pa.field("id", pa.string()),
        pa.field("camera", CameraType),
        pa.field("bbox", BBoxType),
    ]
)

Create a few rows of data following this schema:

In [33]:
rows = [
    {
        "id": "456",
        "camera": camera_example.to_dict(),
        "bbox": BBox.from_xywh([1, 2, 3, 4]).to_dict(),
    },
    {
        "id": "683",
        "camera": camera_example.to_dict(),
        "bbox": BBox.from_xywh([4, 3, 2, 1]).to_dict(),
    },
]

Test saving the table as a lance file and reading it back into memory:


In [34]:
with tempfile.TemporaryDirectory() as temp_dir:
    write_table = pa.Table.from_pylist(rows, schema=schema)

    lance.write_dataset(write_table, uri=Path(temp_dir) / "temp_table.lance")
    db = lancedb.connect(temp_dir)
    lance_table = db.open_table("temp_table")

    read_table = lance_table.to_arrow()

Convert that table back to Python:


In [35]:
read_table.to_pylist()

[{'id': '456',
  'camera': Camera(depth_scale=0.1, cam_K=[1758.377685546875, 0.0, 360.0000000121072, 0.0, 1781.137258093513, 269.9999999622624, 0.0, 0.0, 1.0], cam_R_w2c=[-0.8058909773826599, -0.5643280148506165, -0.17909124493598938, -0.5611616969108582, 0.8244928121566772, -0.0728636085987091, 0.18877841532230377, 0.04177902266383171, -0.9811305999755859], cam_t_w2c=[-10.521206855773926, 40.88941192626953, 1092.1990966796875]),
  'bbox': BBox()},
 {'id': '683',
  'camera': Camera(depth_scale=0.1, cam_K=[1758.377685546875, 0.0, 360.0000000121072, 0.0, 1781.137258093513, 269.9999999622624, 0.0, 0.0, 1.0], cam_R_w2c=[-0.8058909773826599, -0.5643280148506165, -0.17909124493598938, -0.5611616969108582, 0.8244928121566772, -0.0728636085987091, 0.18877841532230377, 0.04177902266383171, -0.9811305999755859], cam_t_w2c=[-10.521206855773926, 40.88941192626953, 1092.1990966796875]),
  'bbox': BBox()}]

Congratulations ! Your custom type is now ready to be used!
